# Part II: Jupiter Simulations
__Created By:__ Caitlin O'Brien <br>
__Edited By:__ Bailey Stephens <br>
__Creation Date:__ 2/1/2022

In [16]:
### Importing necessary libraries
import numpy as np
import pandas as pd
import math as math
import matplotlib.pyplot as plt
import astropy.io.ascii

In [17]:
### Importing necessary functions from packages
from math import pi
from math import sin
from math import sqrt
from astropy.constants import G
from astropy import units as u

## Calculating Detection Signal of Different Methods

We will be calculating detection signal of different methods for the case of a Jupiter-like planet around a Sun-like star. The next three cells define some useful constants relating to our solar system's Earth, Jupiter, and Sun.

In [18]:
# Sun Information
M_s = 1988500e24 * u.kg            # mass (kg)
r_s = 695700 * u.kilometer         # radius (km)
g_s = 274 * (u.meter / (u.s)**2)   # gravity (m/s^2)
L_s = 382.8e24 * (u.J / u.s)       # luminosity (J/s)

In [19]:
# Jupiter Information
M_j = 1898e24 * u.kg               # mass (kg)
r_j = 71492 * u.kilometer          # radius (km)
g_j = 23.1 * (u.meter / (u.s)**2)  # gravity (m/s^2)
d_j = 778.5e6 * u.kilometer        # distance to sun (km)
T_j = 3.742e8 * u.s                # orbital period (s)
v_j = 13.1 * (u.kilometer / u.s)   # orbital velocity (km/s)
i_j = 1.3                          # orbital inclination (degrees)
e_j = 0.049                        # orbital eccentricity

In [20]:
# Earth Information
M_e = 5.97e24 * u.kg               # mass (kg)
r_e = 6378 * u.kilometer           # radius (km)
g_e = 9.8 * (u.meter / (u.s)**2)   # gravity (m/s^2)
d_e = 149.6e6 * u.kilometer        # distance to sun (km)
T_e = 31553280 * u.s               # orbital period (s)
v_e = 29.8 * (u.kilometer / u.s)   # orbital velocity (km/s)
i_e = 0.0                          # orbital inclination (degrees)
e_e = 0.017                        # orbital eccentricity

Next, we define methods to calculate the signals associated with each detection method.

### Radial Velocity
The signal velocity _K_ used to detect exoplanets is given by

\begin{align*}
K &= \left( \frac{M_\text{exo}}{M_\star} \right) \sqrt{\frac{G M_\star}{a}} \sin{i}
\end{align*}

where $G$ is the gravitational constant, $a$ is the semi-major axis of the exoplanet, and $M_\text{exo}$ and $M_\star$ are the masses of the exoplanet and host star respectively. We can then create a function that calculates this _K_ value and plug in the values for our simulated Jupiter.

In [21]:
def K(M_exo,M_star,i,e,T):
    """
    Takes in four values:
        M_exo (Mass of Exoplanet)
        M_star (Mass of host star)
        i (inclination)
        e (eccentricity)
        T (period)
    
    Returns "radial-velocity" signal K
    """
    K = ((2*pi*G)/(T))**(1/3)*((M_exo*sin(i))/(M_exo+M_star)**(2/3))*(1/(sqrt(1-e**2)))
    return K
    
### Prints the K value for our simulated Jupiter
print(K(M_j,M_s,i_j,e_j,T_j))

12.01978785199901 m / s


Hence, the radial velocity signal _K_ = 12.02 m/s for our simulated Jupiter.

### Transit

The first issue with detecting exoplanets using the transit method is that there is a low probability that the exoplanet's orbit falls between the observer and its host star. With the assumption that the exoplanet is much smaller than its host star ($\text{R}_\text{exo} << \text{R}_\star$), this probability is given by

\begin{align*}
    \text{(Transit Probability)} &= \frac{\text{R}_\star}{a} \approx 0.005 \left( \frac{\text{R}_\star}{\text{R}_\odot} \right) \left( \frac{a}{1 \text{ AU}} \right)^{-1}
\end{align*}

for approximately circular orbits. Assuming that this condition is met, then the "signal to noise ratio" (denoted by SNR) is given by

\begin{align*}
    \frac{\text{S}}{\text{N}} &= \frac{\delta}{\sigma_\text{CDPP}} \sqrt{\frac{n_\text{tr} \cdot t_\text{dur}}{3 \text{ hr}}}.
\end{align*}

Both this probability and the SNR for our simulated jupiter are calculated below.

In [24]:
def Prob(r_star,d_star,e):
    """
    Takes in two values:
        r_star (radius of the star)
        d_star (distance to star)
        e (eccentricity of exoplanet orbit)
        
    Produces semimajor axis value, then uses that value to
    produce the probability of the exoplanet transiting in 
    front of its star, and thus if we can observe it.
    """
    
    a = d_star / (1+e)   #calculate semimajor axis
    Prob = r_star / a    #calculate probability
    
    """
    This assumes the following:
            1. The orbit is approximately circular
            2. The radius of the exoplanet << radius of the star
    """
    
    return Prob

### Prints the probability of our jupiter transiting
print(Prob(r_s,d_j,e_j))

0.0009374300578034682


In [15]:
def SNR(R_exo,R_star,d_star,e,P):
    """
    Takes in variables:
        R_exo (radius of exoplanet)
        R_star (radius of star)
        P (period)
    """
    
    a = d_star / (1+e)            #calculate semimajor axis
    delta = R_exo**2 / R_star**2  # calculate photometric depth
    n = P / (7.776e+6*u.s)        #calculate number of transits
    t = (P*R_star) / (pi*a)       #calculate duration of transit
    sigma = 30e-6                 #combined differential photometric precision
    
    # combine into SNR equation
    SNR = (delta/sigma)*sqrt((n*t)/(10800*u.s))
    
    return SNR

### Prints the SNR of our simulated Jupiter
print(SNR(r_j,r_s,d_j,e_j,P_j))

<Quantity 7851.59540871>

Hence, the probability of an observer being able to see this simulated Jupiter is approximately 0.094%, and the SNR produced by this transit would be approximately 7852.

### Direct Imaging

There are two limiting factors that affect the direct imaging method: the Rayleigh limit and the star-planet contrast. The Rayleigh limit describes the minimum angular distance that two stellar objects must be from one another two be able to resolve them as two distinct objects. This equation is given by

\begin{align*}
    \theta \approx 1.22 \frac{\lambda}{D}
\end{align*}

where $\lambda$ is the observation wavelength and D is the diameter of the aperture of the telescope being used. If we assume that the observational wavelength is the beak emission wavelength of a blackbody ($\lambda = 22.3 \mu m$) and the aperture is the largest available on Earth (D = 10.4 m), then we can calculate the minimum angular separation required. We also require that the star-planet contrast _f_ is great enough (typically cited as greater than $f=10^{-7}$). This contrast is given by

\begin{align*}
    f = \left( \frac{R_{p}}{R_\star} \right)^2 \cdot \frac{{\text{exp}\left[\frac{hc}{\lambda k_{B}T_{\star}}\right]} - 1}{\text{exp}\left[\frac{hc}{\lambda k_{B}T_\text{exo}}\right] - 1}
\end{align*}

Hence, we can simulate these requirements for our simulated Jupiter.

In [26]:
# we will be assuming peak emission wavelength of a blackbody
lam = 22.3e-6*u.m
# aperture diameter is approximated by greatest telescope on Earth
D = 10.4*u.m
# we will also be assuming a Jupiter temperature of 130 Kelvin
T_j = 130*u.K

In [27]:
def theta(lamda, diameter):
    """
    Takes in variables:
        lamda (observation wavelength of telescope)
        D (telescope diameter)
    """
    
    ### Calculates the required angular separation
    theta = 1.22*(lamda/diameter)
    
    return theta

### Calculates the Rayleigh limit for simulated Jupiter
print(theta(lam, D))

2.6159615384615386e-06


In [11]:
def f(R_p,R_star,T_star,T_exo):
    """
    Takes in variables:
            R_p (radius of planet)
            R_star (radius of star)
            lam (observation wavelength of telescope)
            T_star (temperature of star)
            T_exo (temperature of exoplanet)
    """
    
    h = 6.62607015e-34*((u.kg*u.m**2*u.s**-2)*u.s)       # Planck's constant
    c = 299792458*(u.m/u.s)                              # speed of light
    k_B = 1.380649e-23*((u.m**2)*(u.kg)/(u.s**2)/(u.K))  # Boltzmann constant
    A = (math.exp((h*c)/(lam*k_B*T_star))-1)             # Numerator of f
    B = (math.exp((h*c)/(lam*k_B*T_exo))-1)              # Denominator of f
    
    ### Calculates the star-planet contrast
    f = ((R_p/R_star)**2)*(A/B)
    return f

### Calculates the star-planet constrast of simulated Jupiter
f(r_j,r_s,T_s,T_j)

<Quantity 8.79368191e-06>

Hence, the simulated must have an angular separation of at least $\theta = 2.62*10^{-6}$ to be visually resolved, and it produces a star-planet constrast of $f=8.79*10^{-6}$.